In [18]:
import numpy as np
import pandas as pd

In [19]:
samples = np.random.multivariate_normal([3,4,5,6],np.diag([1,2,3,4]),size=50000)
w = 100

In [20]:
def emp_mu(window):
    return pd.DataFrame(window).mean().values

def emp_cov(window):
    mu = emp_mu(window)
    cov = np.diag([0,0,0,0])
    
    for i in range(window.shape[0]):
        diff = window[i,:] - mu
        cov = cov + (1/w)*diff.reshape(-1,1).dot(diff.reshape(-1,1).T)
        
    return cov

def emp_cov_2(window):
    mu = emp_mu(window)
    cov = -mu.reshape(-1,1).dot(mu.reshape(-1,1).T)
    
    for i in range(window.shape[0]):
        cov = cov + (1/w)*window[i,:].reshape(-1,1).dot(window[i,:].reshape(-1,1).T)
        
    return cov
    
t = 11000
window_t = samples[t-w:t,:]
window_tp1 = samples[t-w+1:t+1,:]

In [21]:
pd.DataFrame(emp_cov(window_t))

,0,1,2,3
0,0.892441,-0.082808,0.026210,-0.052417
1,-0.082808,2.395898,-0.068706,0.910355
2,0.026210,-0.068706,2.816078,0.428492
3,-0.052417,0.910355,0.428492,5.012212


In [22]:
pd.DataFrame(emp_cov(window_tp1))

,0,1,2,3
0,0.887599,-0.105476,0.025459,-0.078768
1,-0.105476,2.290343,-0.072150,0.785636
2,0.025459,-0.072150,2.815971,0.424226
3,-0.078768,0.785636,0.424226,4.872032


In [23]:
def my_cov_diff(t):
    add_in = (1/w)*samples[t,:].reshape(-1,1).dot(samples[t,:].reshape(-1,1).T) 
    subtract_out = (1/w)*samples[t-w,:].reshape(-1,1).dot(samples[t-w,:].reshape(-1,1).T)
    
    diff = add_in - subtract_out
    
    delta = (1/w)*(samples[t,:] - samples[t-w,:])
    Ert = emp_mu(samples[t-w:t,:])
    
    ch= Ert.reshape(-1,1).dot(delta.reshape(-1,1).T) - delta.reshape(-1,1).dot(Ert.reshape(-1,1).T)
    print(ch)
    
    diff = diff - Ert.reshape(-1,1).dot(delta.reshape(-1,1).T)
    diff = diff - delta.reshape(-1,1).dot(Ert.reshape(-1,1).T)
    diff = diff - delta.reshape(-1,1).dot(delta.reshape(-1,1).T)
    
    return diff

def my_cov_diff_2(t):
    
    Winv = np.diag([1/w,-1/w,-1/w,-1/w,-1/(w**2)])
    
    delta = samples[t,:] - samples[t-w,:]
    mu = emp_mu(samples[t-w:t,:])
    
    L = np.column_stack([samples[t,:], 
                          samples[t-w,:],
                          mu,
                          delta,
                          delta])
    R = np.column_stack([samples[t,:], 
                          samples[t-w,:],
                          delta,
                          mu,
                          delta])
    
    return L.dot(Winv).dot(R.T)

pd.DataFrame(my_cov_diff_2(t) +   emp_cov(window_t))   

,0,1,2,3
0,0.887599,-0.105476,0.025459,-0.078768
1,-0.105476,2.290343,-0.072150,0.785636
2,0.025459,-0.072150,2.815971,0.424226
3,-0.078768,0.785636,0.424226,4.872032
